## import all necessary libraries

In [1]:
!pip install lxml

In [2]:
import requests
import pandas as pd
import numpy as np
import random
import folium
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import lxml

## Foursqaure developer account details

In [3]:
## Foursquare developer account

CLIENT_ID = 'AJ3VPP1DWBF0J12LZGRXTTYPNXHAUS2L4PY5LCP4HQCAPDDN'
CLIENT_SECRET = 'GSD53HFGA5LPASWKWMMJJZLRSUXJ3QA4PMXL51VEGNI4ARP5'
VERSION = '20180604'
LIMIT = 30
print('My_Foursquare credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My_Foursquare credentails:
CLIENT_ID: AJ3VPP1DWBF0J12LZGRXTTYPNXHAUS2L4PY5LCP4HQCAPDDN
CLIENT_SECRET:GSD53HFGA5LPASWKWMMJJZLRSUXJ3QA4PMXL51VEGNI4ARP5


### HTML ADDRESS/SOURCE

In [4]:
html_address = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(html_address).text

### use Beautiful soup for scraping and create a table object with all required information

In [5]:
soup = BeautifulSoup(source, "lxml")
table = soup.find('table', class_="wikitable sortable")

In [6]:
header = table.find_all('th')
columns_list = [x.text.split("\n")[0] for x in header]

### get the rows from the table object and convert to dataframe

In [7]:
body = table.find_all('tr')[1:]
code_list = []
for rows in body:
    row = rows.find_all('td')
    row = [x.text.split("\n")[0] for x in row]
    code_list.append(row)

df = pd.DataFrame(code_list, columns=columns_list)

### remove "Not assigned" from Borough column

In [58]:
df = df[df["Borough"]!= "Not assigned"]
df.head(30)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


### update Neighbourhood to match Borough if Not assigned

In [9]:
neigh = np.where(df["Neighbourhood"] == "Not assigned", df["Borough"], df["Neighbourhood"])
df["Neighbourhood"] = neigh

In [53]:
# grouped = df.groupby(df["Postcode"])["Neighbourhood"].apply(', '.join).reset_index()
grouped = df.groupby(df["Postcode"])
# grouped_df = grouped.merge(df, how="inner", on="Postcode")
grouped_df = grouped["Neighbourhood"].apply(', '.join).reset_index().merge(df, how="inner", on="Postcode")
df_group = grouped_df[["Postcode", "Borough", "Neighbourhood_x"]]
final_df = df_group.drop_duplicates()
final_df.columns = ["Postcode", "Borough", "Neighbourhood"]
final_df.set_index("Postcode", inplace=True)
final_df.reset_index(inplace=True)
final_df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [59]:
final_df.shape

(103, 3)